# Árvores do Takashi

# Modelo 2D

Os resultados aqui apresentados correspondem a um modelo 2D, ou seja, um modelo onde a árvore tem uma largura muito comprida (como se fosse um arbusto que forma uma cerca viva comprida). Com esta homogeneidade na direção perpendicular à página onde estão plotadas as árvores, os resultados iniciais correspondem a uma unidade de comprimento.

O Coeficiente de arrasto é definido como
$$
    C_D = CFx = \frac{F_x}{\frac{1}{2}\rho V^2 A}
$$

Como o problema é 2D, então, $A = L \cdot 1$, de modo que
$$
    C_D = CFx = \frac{F_x}{\frac{1}{2}\rho V^2 L}
$$
e neste caso $F_x$ é a força por unidade de comprimento.


In [ ]:
using Plots
gr(html_output_format=:png)



In [ ]:
include("../src/TreeAerodynamics.jl")


# Velocidade do vento

In [ ]:
Ukm = [100.0, 89.0, 80, 74.0, 60.0, 50.0, 40.0, 28, 20.0]
nvel = length(Ukm)
Uoo = Ukm / 3.3
ρ = 1.1   # Densidade do ar em São Paulo 

# Galho ensaiado

Neste relatório, será empregado o galho de eucalipto ensaiado. Este galho apresentou uma área projetada de aproximadamente 0,5 $m^2$. Aproximando esta área projetada por uma circunferência, isto corresponde a um diâmetro aproximado de 0,8 m.

O coeficiente de arrasto do galho muda com a velocidade. Neste trabalho, o seguinte ajuste de curva foi adotado:

$$
C_D(V) = C_{D,min} + C_{D,0}\cdot\exp(-\alpha\cdot V)
$$

Nesta equação, quando $V\longrightarrow\infty$, $C_D\longrightarrow C_{D,min}$ e para V=0, $C_D = C_{D,min} + C_{D,0}$


In [ ]:
Vtun = [3.3, 4.9, 6.6, 8.2, 9.8, 11.5, 13.1]
Cdtun = [0.94, 0.77, 0.64, 0.56, 0.5, 0.44, 0.41];

In [ ]:
cdwindtunnel = TA.DragCoeff(1.18, 0.32, 0.2)
vvtun = range(3, stop=14, length=100)
cdtun = TA.dragcoeff.(cdwindtunnel, vvtun);

In [ ]:
scatter(Vtun, Cdtun, xlabel="Velocidade do túnel de vento (m/s)", ylabel="Coeficiente de arrasto", label="Cd")
plot!(vvtun, cdtun, label="Ajuste")

# Mudança de tamanho do galho

Não necessariamente o tamanho do galho ensaiado corresponde ao galho que se vai simular. Se o coeficiente de arrasto de um galho é dado pela correlação $C_D^0 = C_D^0(V) = C_D(V, D_0)$, então, para um galho com diâmetro diferente, o coeficiente de arrasto será dado por:

$$
C_D(V, D) = f\left(\frac{D}{D_0}\right) \cdot C_D(V, D_0) = \left(\frac{D}{D_0}\right)^\gamma\cdot C_D(V, D_0)
$$

onde $\gamma \approx 0,45$

Estes são resultados preliminares que precisam ser melhorados e confirmados.

In [ ]:
fcd(d, d₀) = (d/d₀)^0.45

function rescaledrag(Cd::TA.DragCoeff, d, d₀=0.8)
    f = fcd(d, d₀)
    return TA.DragCoeff(Cd.cd0*f, Cd.cdmin*f, Cd.α)
end


In [ ]:
D0 = 0.8

In [ ]:
D = 1.0
Cd = rescaledrag(cdwindtunnel, 1.0, 0.8)
η0 = [0.0, 0.1, 0.25, 0.40, 0.55, 0.7, 0.849, 1.0, 1.15, 1.30, 1.45, 1.6, 1.8, 2, 2.5, 3, 3.5, 4, 5];
x0 = TA.geomseq(0.0, 100*D1, 30, 1.12); 



In [ ]:
Cd

# Árvore 1

In [ ]:
G1 = TA.Branch2d(D, Cd, 0.0, 14.0)
Ux1 = reshape(Uoo, (nvel, 1))
Uy1 = 0.0 * Ux1
Cd1 = TA.dragcoeff.(Cd, Uoo)
CFx1 = Cd1
Fx1 = reshape(Cd1 .* D .* 0.5 .* ρ .* Uoo .^ 2, (nvel,1))
Fxt1 = Fx1[:,1]

In [ ]:
Ux1

In [ ]:
plot(Ukm, Cd1, xlabel="Velocidade (km/h)", ylabel="CFx")


In [ ]:
using CSV
using DataFrames

In [ ]:
df = DataFrame(U=Ukm, Uoo=Uoo, CFx=CFx1, Fx=Fxt1)
CSV.write("temp.txt", df; delim=";")

In [ ]:
plot(Ukm, Fxt1, xlabel="Velocidade (km/h)", ylabel="Força por unidade de comprimento (N/m)", 
label="Fx Árvore 1")

In [ ]:
function simulatevels(branches, Uoo, x, η; err=1e-5, rlx=0.2)
    
    nu = length(Uoo)
    nb = length(branches)
    drag = []
    
    println("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    println("Vel: ", Uoo[1])
    push!(drag, TA.velinterference(branches, (x,y)->(Uoo[1], 0.0), x, η; err=err, rlx=rlx))
    
    for i = 2:nu
    println("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    println("Vel: ", Uoo[i])
        fun = (x,y) -> (Uoo[i], 0.0)
        Ux = drag[i-1][1] .* Uoo[i]/Uoo[i-1]
        Uy = drag[i-1][1] .* Uoo[i]/Uoo[i-1]
        rlx = min(0.5, rlx*1.5)
        push!(drag, TA.velinterference(branches, fun, x, η, err=err, rlx=rlx, Uxinit=Ux, Uyinit=Uy))
    end
    
    return drag
    
    
end

In [ ]:
function forcetot(branches, Ux, Uy, ρ=1.2)

    Fx = 0.0
    Fy = 0.0
    nb = length(branches)
    for i = 1:nb
        ux = Ux[i]
        uy = Uy[i]
        U = hypot(ux, uy)
        Cd = TA.dragcoeff(branches[i], U)
        D = TA.diameter(branches[i])
        α = atan(uy, ux)
        cα = cos(α)
        sα = sin(α)
        fi = 0.5 * ρ * Cd * D * U^2
        
        Fx += fi*cα
        Fy += fi*sα
    end

    return Fx, Fy
end

function force(branch, ux, uy, ρ=1.2)

    U = hypot(ux, uy)
    Cd = TA.dragcoeff(branch, U)
    D = TA.diameter(branch)
    α = atan(uy, ux)
    cα = cos(α)
    sα = sin(α)
    fi = 0.5 * ρ * Cd * D * U^2
        
    Fx = fi*cα
    Fy = fi*sα
    

    return Fx, Fy
end


function forcecoef(branches, Ux, Uy, Uoo, Lref)
    ρ = 1.0
    Fx, Fy = forcetot(branches, Ux, Uy, 1.0)
    CFx = Fx / (0.5 * ρ * Lref * Uoo^2)
    CFy = Fy / (0.5 * ρ * Lref * Uoo^2)    
    
    return CFx, CFy
end



# Árvore 2

In [ ]:
G1 =  TA.Branch2d(D, Cd, 0.0, 14.0)
G2 =  TA.Branch2d(D, Cd, 7.0, 7.0)
G3 =  TA.Branch2d(D, Cd, -7.0, 7.0)
T2 = [G1, G2, G3]
L2 = 14.0

In [ ]:
drag2 = simulatevels(T2, Uoo, x0, η0)

In [ ]:
Ux2 = hcat([d[1] for d in drag2]...)
Uy2 = hcat([d[2] for d in drag2]...);
Cd2 = hcat([d[3] for d in drag2]...);
CFx2 = [forcecoef(T2 , Ux2[:,i], Uy2[:,i], Uoo[i], L2)[1] for i in 1:length(Uoo)];
xc2 = [b.xc for b in T2];
yc2 = [b.yc for b in T2];


In [ ]:
scatter(xc2, yc2, color=:green, aspect_ratio=1.0, legend=false, markersize=20)
plot!([0, 0], [0, 14], linewidth=3, color=:brown)
plot!([-7, 7], [7, 7], linewidth=3, color=:brown)
i1 = 1
i2 = length(Uoo)
quiver!(xc2, yc2, gradient=(3Ux2[:,i1]/Uoo[i1], 2Uy2[:,i1]/Uoo[i1]))
quiver!(xc2, yc2, gradient=(3Ux2[:,i2]/Uoo[i2], 2Uy2[:,i2]/Uoo[i2]))

In [ ]:
Ux2


In [ ]:
Uy2

In [ ]:
plot(Ukm, CFx2, label="Lref = 14 m", xlabel="Velocidade (km/h)", 
    ylabel="Cd")

## Comparação com um galho único

In [ ]:
plot(Ukm, CFx2 * 14 ./ (CFx1*length(T2)), xlabel="Velocidade (km/h)", 
    ylabel="Razão entre CFx", title="Razão entre força total e Galhos isolados", label="Árvore 2")



# Árvore 3

In [ ]:
G1 =  TA.Branch2d(D, Cd, 0.0, 14.0)
G2 =  TA.Branch2d(D, Cd, 7.0, 7.0)
G3 =  TA.Branch2d(D, Cd, 4.95, 11.95)
G4 =  TA.Branch2d(D, Cd, -4.95, 11.95)
G5 =  TA.Branch2d(D, Cd, -7.0, 7.0)
T3= [G1, G2, G3, G4, G5]
L3= 14.0

In [ ]:
drag3 = simulatevels(T3, Uoo, x0, η0)

In [ ]:
Ux3 = hcat([d[1] for d in drag3]...)
Uy3 = hcat([d[2] for d in drag3]...);
Cd3 = hcat([d[3] for d in drag3]...);
CFx3 = [forcecoef(T3 , Ux3[:,i], Uy3[:,i], Uoo[i], L3)[1] for i in 1:length(Uoo)];
xc3 = [b.xc for b in T3];
yc3 = [b.yc for b in T3];


In [ ]:
scatter(xc3, yc3, color=:green, aspect_ratio=1.0, legend=false, markersize=20)
plot!([0, 0], [0, 14], linewidth=3, color=:brown)
plot!([-7, 7], [7, 7], linewidth=3, color=:brown)
plot!([0, 4.95], [7, 11.95], linewidth=3, color=:brown)
plot!([0, -4.95], [7, 11.95], linewidth=3, color=:brown)

i1 = 1
i2 = length(Uoo)
quiver!(xc3, yc3, gradient=(2Ux3[:,i1]/Uoo[i1], 2Uy3[:,i1]/Uoo[i1]))
quiver!(xc3, yc3, gradient=(2Ux3[:,i2]/Uoo[i2], 2Uy3[:,i2]/Uoo[i2]))

In [ ]:
Ux3


In [ ]:
Uy3

In [ ]:
plot(Ukm, CFx3, label="Lref = 14 m", xlabel="Velocidade (km/h)", 
    ylabel="Cd")

## Comparação com um galho único

In [ ]:
plot(Uoo, CFx3 * 14 ./ (CFx1*length(T3)), xlabel="Velocidade (km/h)", 
    ylabel="Razão entre CFx", title="Razão entre força total e Galhos isolados", label="Árvore 3")


# Árvore 4

In [ ]:
G1 =  TA.Branch2d(D, Cd, 0.0, 14.0)
G2 =  TA.Branch2d(D, Cd, 7.0, 7.0)
G3 =  TA.Branch2d(D, Cd, 6.06, 10.05)
G4 =  TA.Branch2d(D, Cd, 3.5, 13.06)
G5 =  TA.Branch2d(D, Cd, -3.5, 13.06)
G6 =  TA.Branch2d(D, Cd, -6.06, 10.05)
G7 =  TA.Branch2d(D, Cd, -7.0, 7.0)
T4= [G1, G2, G3, G4, G5, G6, G7]
L4= 14.0

In [ ]:
drag4 = simulatevels(T4, Uoo, x0, η0)

In [ ]:
Ux4 = hcat([d[1] for d in drag4]...)
Uy4 = hcat([d[2] for d in drag4]...);
Cd4 = hcat([d[3] for d in drag4]...);
CFx4 = [forcecoef(T4 , Ux4[:,i], Uy4[:,i], Uoo[i], L4)[1] for i in 1:length(Uoo)];
xc4 = [b.xc for b in T4];
yc4 = [b.yc for b in T4];


In [ ]:
scatter(xc4, yc4, color=:green, aspect_ratio=1.0, legend=false, markersize=20)
plot!([0, 0], [0, 14], linewidth=3, color=:brown)
plot!([-7, 7], [7, 7], linewidth=3, color=:brown)
plot!([0, 6.06], [7, 10.5], linewidth=3, color=:brown)
plot!([0, -6.06], [7, 10.5], linewidth=3, color=:brown)
plot!([0, 3.5], [7, 13.06], linewidth=3, color=:brown)
plot!([0, -3.5], [7, 13.06], linewidth=3, color=:brown)

i1 = 1
i2 = length(Uoo)
quiver!(xc4, yc4, gradient=(3Ux4[:,i1]/Uoo[i1], 3Uy4[:,i1]/Uoo[i1]))
quiver!(xc4, yc4, gradient=(3Ux4[:,i2]/Uoo[i2], 3Uy4[:,i2]/Uoo[i2]))

In [ ]:
Ux4

In [ ]:
Uy4

In [ ]:
plot(Ukm, CFx4, label="Lref = 14 m", xlabel="Velocidade (km/h)", 
    ylabel="Cd")

## Comparação com um galho único

In [ ]:
plot(Ukm, CFx4 * 14 ./ (CFx1*length(T4)), xlabel="Velocidade (km/h)", 
    ylabel="Razão entre CFx", title="Razão entre força total e Galhos isolados", label="Árvore 4")



# Extrapolação para 3D

A análise neste trabalho foi 2D. Naturalmente, deseja-se de alguma maneira   aplicar estes resultados para o caso real - 3D. 

Os coeficientes de arrasto para as diferentes aŕvores (CFx1, CFx2, CFx3, CFx4) foram obtidos considerando a largura da copa que vale, com a exceção  da árvore 1, D = 14 m:

$$
CFx = \frac{Fxu}{\frac{1}{2}\rho V^2 D}
$$

onde Fxu é a força por unidade de comprimento.


Lembrando que os resultados anteriores correspondem a carreiras longas de árvores (lembre-se da cerca viva...), a primeira tentação seria simplesmente multiplicar a força por unidade de comprimento pelo comprimento $D$. Esta abordagem tem dois problemas:

 * A copa, vista de cima, tende a ser circular e esta abordagem apresenta uma copa muito maior
 * No caso 3D, além de se desviar por cima (o que foi simulado neste modelo) o ar pode ser desviar pelos lados.
 
 Assim, o coeficiente de força 3D será menor que o 2D por um fator de 2 - 3.